# NL2SQL Practice 2

In [ ]:
!unzip /content/data.zip -d /content

In [ ]:
pip install -r requirements.txt

## OpenAI API Connection

In [ ]:
from getpass import getpass
from openai import OpenAI

import chromadb
import chromadb.utils.embedding_functions as embedding_functions

import os
import json
import pandas as pd
from tqdm import tqdm

OPENAI_API_KEY = getpass("Please enter your OpenAI API key: ")

llm = OpenAI(api_key=OPENAI_API_KEY)

## Storing Data in Knowledge Databases (VectorDB)

In [ ]:
import json
import chromadb
from chromadb.utils import embedding_functions

# Load schema information file
with open("superhero.json", "r", encoding="utf-8") as f:
    schema_data = json.load(f)

# Prepare documents and metadata
documents = [] # list of contents that will be transformed into embedding vectors ("{column} + {description}")
metadatas = [] # list of metadata that will not be transformed into embedding vectors, but still linked with the embedding vectors
ids = [] # (id for each data stored in vectordb should be in "text" format for current version of ChromaDB)

for table in schema_data:
    table_name = table["table"]
    for col in table["column"]:
        col_name = col["column_name"]
        description = col.get("description", "").lstrip("#").strip()
        document = f"column_name: {col_name}\n{description}"
        # e.g., "column_name: colour The colour column in the superhero table represents the color of the superhero's skin/eye/hair/etc."

        documents.append(document)
        metadatas.append({
            "table": table_name,
            "column": col_name
        })
        ids.append(f"col-{table_name}-{col_name}")

# ChromaDB setup
client = chromadb.PersistentClient(path="./vectordb/")

# Use OpenAI embedding model for embedding vector transformation.
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)

# Create collection
collection = client.get_or_create_collection("column_description", embedding_function=openai_ef)

# Save the data in collection
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print("Column descriptions saved to ChromaDB!")

In [ ]:
# Json file storing few-shot exmamples
jsonl_file = "fewshot.jsonl"

# Open Json file -> Pandas DataFrame
records = []
with open(jsonl_file, 'r', encoding='utf-8') as f:
    for line in f:
        records.append(json.loads(line))

df = pd.DataFrame(records)

client = chromadb.PersistentClient(path="./vectordb/")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"
)

collection = client.get_or_create_collection("fewshot", embedding_function=openai_ef)

documents = df["question"].tolist() #
metadatas = df[["evidence", "question", "SQL"]].to_dict(orient="records")
ids = ["few-shot-" + str(qid) for qid in df["question_id"]]

collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print("All few-shot examples saved in ChromaDB!")

## Text-to-SQL using OpenAI LLM (GPT-4o-mini)

In [ ]:
# user_question / gold SQL from BIRD dev.
with open("sample.json", "r", encoding="utf-8") as f:
    sample_data = json.load(f)

# first sample_data (sample_data[1]: second sample_data)
user_question = sample_data[0]['question']
knowledge_evidence = sample_data[0]['evidence']
gold_SQL = sample_data[0]['SQL'] # for evaluation

print("Question: ", user_question)
print("Hint: ", knowledge_evidence)
print("Gold SQL: ", gold_SQL)

### Schema Linking (Get relevant columns)

In [ ]:
def find_relevant_columns(query, top_k=5):
    client = chromadb.PersistentClient(path="./vectordb/")

    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=OPENAI_API_KEY,
        model_name="text-embedding-3-small" # use same embedding model with one that were used for storing data in vectorDB
    )

    collection = client.get_or_create_collection("column_description", embedding_function=openai_ef)

    # Similarity Search
    results = collection.query(
        query_texts=[query], # batch available
        n_results=top_k # retrieve top k relevant data
    )

    print(f"\n🔍 Top {top_k} columns relevant to:\n\"{query}\"\n")

    for doc, meta, score in zip(results['documents'][0], results['metadatas'][0], results['distances'][0]):
        print(f"# Table: {meta['table']}, Column: {meta['column']}")
        print(f"# Description: {doc.splitlines()[1]}")
        print(f"# Score: {score:.4f}\n")

    return results['metadatas'][0]  # 리스트 of {table, column}

In [ ]:
relevant_columns = find_relevant_columns(user_question, top_k=5)
print(relevant_columns)

### Schema Representation

In [ ]:
import json
import json

def generate_filtered_schema(input_file, used_columns=None, include_descriptions=False):
    with open(input_file, "r", encoding="utf-8") as f:
        tables = json.load(f)

    output_lines = []

    if used_columns: # use schema selected from schema-linking
        used_set = set((item["table"], item["column"]) for item in used_columns)
    else:
        used_set = None  # use full schema instead of selected subset

    for table in tables:
        table_name = table["table"]

        if used_set:
            table_columns = [col for col in table["column"] if (table_name, col["column_name"]) in used_set]
            if not table_columns:
                continue
        else:
            table_columns = table["column"]

        output_lines.append(f"Table: {table_name}")

        for col in table_columns:
            col_name = col["column_name"]
            is_pk = col["PK"] == 1
            fk = col["FK"]
            desc = col.get("description", None)

            if is_pk:
                output_lines.append(f"- Column (PK): {col_name}")
            elif fk:
                output_lines.append(f"- Column (FK): {col_name}")
            else:
                output_lines.append(f"- Column: {col_name}")

            # include column descriptions in prompt?
            if include_descriptions:
                if desc:
                    cleaned = desc.strip().lstrip("#").strip()
                    output_lines.append(f"  - Description: {cleaned}")

        output_lines.append("")

    return "\n".join(output_lines)

In [ ]:
relevant_columns = find_relevant_columns(user_question, top_k=5)

schema_info = generate_filtered_schema(
    input_file="superhero.json",
    used_columns=None, # relevant_columns
    include_descriptions=True #
)

print(schema_info)

### Few-shot Retrieval (Get relevant examples)

In [ ]:
def get_relevant_fewshots(user_question, top_k=3):
    import chromadb
    from chromadb.utils import embedding_functions

    client = chromadb.PersistentClient(path="./vectordb/")
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=OPENAI_API_KEY,
        model_name="text-embedding-3-small"
    )
    collection = client.get_or_create_collection("fewshot", embedding_function=openai_ef)

    results = collection.query(
        query_texts=[user_question],
        n_results=top_k
    )

    fewshot_blocks = []
    for idx, (doc, meta) in enumerate(zip(results["documents"][0], results["metadatas"][0]), 1):
        block = f"""### Example {idx}
Question: {meta['question']}
Evidence: {meta['evidence']}
SQL:
{meta['SQL']}"""
        fewshot_blocks.append(block)

    return "\n\n".join(fewshot_blocks)

fewshots_text = get_relevant_fewshots(user_question, top_k=3)

print(fewshots_text)

### Prompt Template for SQL Generation

In [ ]:
prompt_template = """You are a data science expert.
Below, you are presented with a database schema and a question.
Your task is to read the schema, understand the question, and generate a valid SQLite query to answer the question.
Before generating the final SQL query think step by step on how to write the query.

Database Schema:
{DATABASE_SCHEMA}

Few-shot Examples:
{FEWSHOTS}

This schema offers an in-depth description of the database’s architecture, detailing tables, columns, primary keys, foreign keys, and any pertinent information regarding relationships or constraints. Special attention should be given to the examples listed beside each column, as they directly hint at which columns are relevant to our query.

Database admin instructions:
- Make sure you only output the information that is asked in the question. If the question asks for a specific column, make sure to only include that column in the SELECT clause, nothing more.
- Predicted query should return all of the information asked in the question without any missing or extra information.

Question:
{QUESTION}

Hint:
{HINT}

Please respond with a JSON object structured as follows:

{{
  "chain_of_thought_reasoning": "Your thought process on how you arrived at the final SQL query.",
  "SQL": "Your SQL query in a single string."
}}

Priority should be given to columns that have been explicitly matched with examples relevant to the question’s context.

Take a deep breath and think step by step to find the correct SQLite SQL query.
"""

### Generate SQL query using LLM

In [ ]:
prompt = prompt_template.format(
    DATABASE_SCHEMA=schema_info,
    QUESTION=user_question,
    HINT=knowledge_evidence, # ""
    FEWSHOTS=fewshots_text, # ""
)

response = llm.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

generated_text = response.choices[0].message.content
generated_dict = json.loads(generated_text)
predicted_SQL = generated_dict["SQL"]

In [ ]:
import sqlite3

db_path = "superhero.sqlite"

def run_query(db_path, SQL):
  try:
      conn = sqlite3.connect(db_path)
      cursor = conn.cursor()

      cursor.execute(SQL)
      results = cursor.fetchall()

      # # PRINT RESULT (MIGHT BE REALLY LONG!)
      # print("Results:")
      # for row in results:
      #     print(row)

      return results
  except Exception as e:
      print("Error while executing query:", e)
      return None
  finally:
      conn.close()

In [ ]:
def normalize_result(rows):
    # set comparison ignoring row/column order
    return set(tuple(row) for row in rows)

predicted_results = run_query(db_path, predicted_SQL)
gold_results = run_query(db_path, gold_SQL)

if predicted_results is not None and gold_results is not None:
    if normalize_result(predicted_results) == normalize_result(gold_results):
        print("Prediction matches gold SQL output!")
    else:
        print("Prediction does NOT match gold SQL output.")
        print("Predicted Results:", predicted_results)
        print("Gold Results:", gold_results)